In [1]:
from paddleocr import PaddleOCR, draw_ocr
import os,glob,csv,math,json,re,math,cv2,slideio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from IPython.display import display, HTML
from fuzzywuzzy import fuzz
from utils import show_images
from tqdm import tqdm

In [2]:
# Setup model
ocr_model = PaddleOCR(lang='en',use_angle_cls = False,show_log=False)

In [3]:
import logging
from ppocr.utils.logging import get_logger as ppocr_get_logger
ppocr_get_logger().setLevel(logging.ERROR)

In [4]:
# Open our folder and list the number of files inside
folder_name,type = 'A20-169','svs' # Specify the folder name (first param) and file type (second param)
folder = glob.glob(f"{folder_name}/*.{type}")
print(f"{len(folder)} {type} files identified.")

72 svs files identified.


In [5]:
columns = ['participant_id', 'stain_id', 'brain_region','label']
csv_filename = f"{folder_name}.csv"  # Change this to your desired file name

with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(columns)

In [6]:
label_count = 0

for file in tqdm(folder):
    slide = slideio.open_slide(file)
    scene = slide.get_scene(0)
    # print(scene)
    # This part gets the image per slide
    image_names = slide.get_aux_image_names()
    images = []
    for name in image_names:
        if name == 'Label':
            image = slide.get_aux_image_raster(name)
            images.append(image)  
            rotated_image = np.rot90(image, k=-1) # Capture a rotated image of the label
            result = ocr_model.ocr(rotated_image) # Process rotated image with Paddle
            # Build a list containing all chunks identified by Paddle 
            text_list = []
            for result_group in result:
                for text_region in result_group:
                    text, confidence = text_region[1]  # Extract text and confidence
                    # print("Text:", text); print("Confidence:", confidence)
                    text_list.append(text)
                    # Coordinates are stored in text_region[0] if needed
            garbage = ['starfrost', 'adrc']  # Clean the result a little
            text_list = [thing for thing in text_list if thing.lower() not in garbage]
            # Phase 1: Gather the Participant ID:
            participant_pattern = r'([A-Z0-9]{3})-([A-Z0-9]{3})'
            participant_matches = [re.search(participant_pattern, i) for i in text_list if re.search(participant_pattern, i)]
            participant_id = [match[0] for match in participant_matches]
            participant_id = participant_id[0]; print(f"Participant ID: {participant_id}")
            # Phase 2: Gather the Stain ID:
            predefined_choices = ['LB509', 'HE', 'PHF-1', 'TDP-43', '10D5'] # Stain Choices
            stain_id = None
            best_similarity = 0
            for i in text_list:
                for choice in predefined_choices:
                    similarity = fuzz.ratio(i, choice)  # Calculate Lebenshtein distance
                    if similarity > best_similarity:
                        best_similarity = similarity
                        stain_id = choice
            print(f"Stain ID: {stain_id}")
            # Phase 2.5 (cleaning):
            prune = [item for item in text_list if item.lower() not in (stain_id.lower(), participant_id[0].lower())]
            text_list = prune # Clean the result a bit more 2 of 3
            # Phase 3: Gather the Brain Region ID:
            brain_region_pattern = r'\bL\d+[A-Z]?\b'
            brain_region_matchs = [re.search(brain_region_pattern, i) for i in text_list if re.search(brain_region_pattern, i)]
            brain_region = [match[0] for match in brain_region_matchs]
            brain_region = brain_region[0]; print(f"Brain region match: {brain_region}")
            # Phase 3.5 (cleaning):
            prune = [item.replace(brain_region[0], "").strip() for item in text_list]
            text_list = prune  # Clean the result a bit more 3 of 3
            # Phase 4 Print Results: 
            label_count += 1
            # Write the rows to the CSV file
            with open(f'{folder_name}.csv', mode='a', newline='') as csv_file:
                csv_writer = csv.writer(csv_file)
                label = f"{participant_id}_{brain_region}_{stain_id}"
                csv_writer.writerow([participant_id, stain_id, brain_region, label])
            # Phase 5 Rename Each File:
            new_name = os.path.join(os.path.dirname(file), label + ".svs")
            os.rename(file, new_name)

  1%|█▎                                                                                              | 1/72 [00:02<02:23,  2.02s/it]

Participant ID: A20-169
Stain ID: PHF-1
Brain region match: L3


  3%|██▋                                                                                             | 2/72 [00:03<01:46,  1.52s/it]

Participant ID: A20-169
Stain ID: LB509
Brain region match: L3


  4%|████                                                                                            | 3/72 [00:04<01:46,  1.54s/it]

...

 67%|███████████████████████████████████████████████████████████████▎                               | 48/72 [00:59<00:29,  1.23s/it]

Participant ID: A20-169
Stain ID: TDP-43
Brain region match: L12


 67%|███████████████████████████████████████████████████████████████▎                               | 48/72 [01:00<00:30,  1.26s/it]

Participant ID: A20-169
Stain ID: HE


IndexError: list index out of range